In [1]:
!pip install torch torchvision torchaudio
!pip install git+https://github.com/openai/CLIP.git
!pip install pillow
import requests


You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


  Cloning https://github.com/openai/CLIP.git to c:\users\asus\appdata\local\temp\pip-req-build-s12ih_td
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1


  Running command git clone -q https://github.com/openai/CLIP.git 'C:\Users\ASUS\AppData\Local\Temp\pip-req-build-s12ih_td'
You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("img.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.12837298 0.29087263 0.5807544 ]]


In [3]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

           snake: 65.31%
          turtle: 12.29%
    sweet_pepper: 3.83%
          lizard: 1.88%
       crocodile: 1.75%


In [4]:
!pip install pandas

You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install faiss-cpu
!pip install neo4j

You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import torch
import clip
from PIL import Image
import pandas as pd
import faiss

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


df = pd.read_csv("products.csv")


def get_clip_embedding(row):
    text = f"{row['product_name']}.{row['category']}.{row['description']}"
    image_path = row['image_path']
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text_tokens = clip.tokenize([text]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_tokens)
        combined = (image_features + text_features) / 2
        return combined.cpu().numpy()

embeddings = [get_clip_embedding(row) for idx, row in df.iterrows()]

import numpy as np

dimension = embeddings[0].shape[1]
index = faiss.IndexFlatL2(dimension)

# Wrap with metadata
product_ids = df['product_id'].tolist()
index_with_ids = faiss.IndexIDMap(index)
index_with_ids.add_with_ids(np.vstack(embeddings), np.array(product_ids))

from neo4j import GraphDatabase

# Neo4j connection (driver)
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "secretgraph"


driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
# Clear existing data
driver.execute_query("MATCH (n) DETACH DELETE n")
# Insert products into Neo4j
for idx, row in df.iterrows():
    driver.execute_query(
        """
        MERGE (p:Product {productId: $pid, name: $name})
        MERGE (c:Category {name: $category})
        MERGE (pr:Price {value: $price})
        MERGE (img:Image {path: $imagePath})

        MERGE (p)-[:BELONGS_TO]->(c)
        MERGE (p)-[:HAS_PRICE]->(pr)
        MERGE (p)-[:HAS_IMAGE]->(img)
        """,
        pid=int(row['product_id']),
        name=row['product_name'],
        category=row['category'],
        price=float(row['price']),
        imagePath=row['image_path']
    )

def embed_query(text_query):
    text_tokens = clip.tokenize([text_query]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
    return text_features.cpu().numpy()

def search_faiss(query_embedding, k=5):
    D, I = index_with_ids.search(query_embedding, k)
    return I[0].tolist()  # List of productIds

def fetch_product_info(product_ids):
    results = []
    for pid in product_ids:
        record = driver.execute_query(
            """
            MATCH (p:Product {productId: $pid})
            OPTIONAL MATCH (p)-[:BELONGS_TO]->(c:Category)
            OPTIONAL MATCH (p)-[:HAS_PRICE]->(pr:Price)
            OPTIONAL MATCH (p)-[:HAS_IMAGE]->(img:Image)
            RETURN p.name AS name, p.productId AS id,
                   c.name AS category, pr.value AS price,
                   img.path AS imagePath
            """,
            pid=int(pid)
        )
        if record and record.records:
            results.append(record.records[0].data())
    return results

def semantic_product_search(user_query):
    query_embedding = embed_query(user_query)
    product_ids = search_faiss(query_embedding)
    product_info = fetch_product_info(product_ids)
    return product_info

results = semantic_product_search("soft breathable cotton shirt")
for r in results:
    print(f"Product: {r['name']}, Category: {r['category']}, Price: ₹{r['price']}, Image: {r['imagePath']}")

Product: Classic White T-Shirt, Category: Clothing, Price: ₹12.99, Image: images_prod/tshirt.webp
Product: Running Shoes, Category: Footwear, Price: ₹59.99, Image: images_prod/shoes.webp
Product: Smartphone 128GB, Category: Electronics, Price: ₹699.0, Image: images_prod/smartphone.webp
Product: Blue Denim Jeans, Category: Clothing, Price: ₹39.99, Image: images_prod/jeans.avif
Product: Organic Green Tea, Category: Groceries, Price: ₹8.5, Image: images_prod/greentea.jpg


In [7]:
import requests

query = "kettle"
access_key = "l50eZBFQwQl3vPKpvi-gqEcyByO90xRo29bmb2C_wUk"

url = f"https://api.unsplash.com/search/photos?query={query}&client_id={access_key}"
response = requests.get(url).json()

# Get first image URL
image_url = response["results"][0]["urls"]["regular"]
print("Image URL:", image_url)


Image URL: https://images.unsplash.com/photo-1594213114663-d94db9b17125?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w4MTIyOTZ8MHwxfHNlYXJjaHwxfHxrZXR0bGV8ZW58MHx8fHwxNzYwMDAxNTExfDA&ixlib=rb-4.1.0&q=80&w=1080


In [2]:
import re



In [3]:
import requests
import os

def download_image(url, folder_path, filename):
   
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, filename)
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Image downloaded successfully to: {file_path}")
        return file_path
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {url}: {e}")
        return None

# Example usage:

output_folder = "downloaded_images"



In [8]:
import torch
import clip
from PIL import Image
import pandas as pd
import faiss
import requests
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


df = pd.read_csv("products.csv")


def get_clip_embedding(row):
    text = f"{row['product_name']}.{row['category']}.{row['description']}"
    image_path = row['image_path']
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text_tokens = clip.tokenize([text]).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_tokens)
        combined = (image_features + text_features) / 2
        return combined.cpu().numpy()

embeddings = [get_clip_embedding(row) for idx, row in df.iterrows()]

import numpy as np

dimension = embeddings[0].shape[1]
index = faiss.IndexFlatL2(dimension)

# Wrap with metadata
product_ids = df['product_id'].tolist()
index_with_ids = faiss.IndexIDMap(index)
index_with_ids.add_with_ids(np.vstack(embeddings), np.array(product_ids))

from neo4j import GraphDatabase

# Neo4j connection (driver)
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "secretgraph"


import requests


driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
# Clear existing data
driver.execute_query("MATCH (n) DETACH DELETE n")
# Insert products into Neo4j
for idx, row in df.iterrows():
    query = row['product_name']
    access_key = "l50eZBFQwQl3vPKpvi-gqEcyByO90xRo29bmb2C_wUk"
    query = re.sub(r'[^a-zA-Z0-9_-]', '_', query)
    
    url = f"https://api.unsplash.com/search/photos?query={query}&client_id={access_key}"
    response = requests.get(url).json()
    image_filename = f"{query}.jpg"
    
    image_url = response["results"][0]["urls"]["regular"]
    downloaded_image_path = download_image(image_url, output_folder, image_filename)
    
    driver.execute_query(
        """
        MERGE (p:Product {productId: $pid, name: $name})
        MERGE (c:Category {name: $category})
        MERGE (pr:Price {value: $price})
        MERGE (img:Image {path: $imagePath})

        MERGE (p)-[:BELONGS_TO]->(c)
        MERGE (p)-[:HAS_PRICE]->(pr)
        MERGE (p)-[:HAS_IMAGE]->(img)
        """,
        pid=int(row['product_id']),
        name=row['product_name'],
        category=row['category'],
        price=float(row['price']),
        imagePath=downloaded_image_path
    )

def embed_query(text_query):
    text_tokens = clip.tokenize([text_query]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
    return text_features.cpu().numpy()

def search_faiss(query_embedding, k=5):
    D, I = index_with_ids.search(query_embedding, k)
    return I[0].tolist()  # List of productIds

def fetch_product_info(product_ids):
    results = []
    for pid in product_ids:
        record = driver.execute_query(
            """
            MATCH (p:Product {productId: $pid})
            OPTIONAL MATCH (p)-[:BELONGS_TO]->(c:Category)
            OPTIONAL MATCH (p)-[:HAS_PRICE]->(pr:Price)
            OPTIONAL MATCH (p)-[:HAS_IMAGE]->(img:Image)
            RETURN p.name AS name, p.productId AS id,
                   c.name AS category, pr.value AS price,
                   img.path AS imagePath
            """,
            pid=int(pid)
        )
        if record and record.records:
            results.append(record.records[0].data())
    return results

def semantic_product_search(user_query):
    query_embedding = embed_query(user_query)
    product_ids = search_faiss(query_embedding)
    product_info = fetch_product_info(product_ids)
    return product_info

results = semantic_product_search("soft breathable cotton shirt")
for r in results:
    print(f"Product: {r['name']}, Category: {r['category']}, Price: ₹{r['price']}, Image: {r['imagePath']}")

Image downloaded successfully to: downloaded_images\Classic_White_T-Shirt.jpg
Image downloaded successfully to: downloaded_images\Blue_Denim_Jeans.jpg
Image downloaded successfully to: downloaded_images\Wireless_Bluetooth_Headphones.jpg
Image downloaded successfully to: downloaded_images\Wooden_Coffee_Table.jpg
Image downloaded successfully to: downloaded_images\Organic_Green_Tea.jpg
Image downloaded successfully to: downloaded_images\Running_Shoes.jpg
Image downloaded successfully to: downloaded_images\Smartphone_128GB.jpg
Image downloaded successfully to: downloaded_images\Office_Chair.jpg
Image downloaded successfully to: downloaded_images\Leather_Backpack.jpg
Image downloaded successfully to: downloaded_images\Electric_Kettle.jpg
Product: Classic White T-Shirt, Category: Clothing, Price: ₹12.99, Image: downloaded_images\Classic_White_T-Shirt.jpg
Product: Running Shoes, Category: Footwear, Price: ₹59.99, Image: downloaded_images\Running_Shoes.jpg
Product: Smartphone 128GB, Category:

In [9]:
!pip install langchain langchain-ollama langchain-chroma pandas

You should consider upgrading via the 'C:\Users\ASUS\OneDrive\Desktop\graphdb\final\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate


model1 = OllamaLLM(model="llama3.2:1b")

template = """
Here are some relevant nodes: {reviews}

Here is the question to answer: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model1

while True:
    print("\n\n-------------------------------")
    question = input("Ask your question (q to quit): ")
    print("\n\n")
    if question == "q":
        break
    
    reviews = semantic_product_search(question)
    result = chain.invoke({"reviews": reviews, "question": question})
    print(result)



-------------------------------


Ask your question (q to quit):  electronics





It appears you're trying to find products that belong to the "Electronics" category.

Looking at your list of nodes, you can see that there are several products with a "category" value of "Electronics". However, I'd like to clarify what exactly you mean by "Electronics".

From your list, all four products seem to fall under the Electronics category:

1. Smartphone 128GB
2. Wireless Bluetooth Headphones
3. Office Chair

If you're looking for more products in this category, I can help you search further or provide suggestions based on your existing nodes.


-------------------------------


Ask your question (q to quit):  q
